# Setup

In [ ]:
!pip install --upgrade fastai
!pip install --upgrade torch==1.1.0

In [ ]:
import os
import random
from copy import deepcopy
import shutil
from fastai.vision import *
from pathlib import Path
from fastai.widgets import *

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
np.random.seed(42)
bs = 64
path = Path("./data/images/processed")

# Load Data

In [ ]:
def LoadData(path, bs=64, size=224, valid="valid"):
    return (ImageList.from_folder(path)    
            .split_by_rand_pct(0.2, 42)
            .label_from_folder()
            .transform(get_transforms(), size=size) 
            .databunch(bs=bs)).normalize(imagenet_stats)

# resnet50 - 256

In [ ]:
data = LoadData(path, bs, 256)
learn = cnn_learner(data, models.resnet50, metrics=[accuracy, error_rate])

In [ ]:
learn.fit_one_cycle(20)
learn.save('stage-1')

# resnet50 - 256 - learning rate + unfreeze

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.load("stage-1");

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(4.79e-6, 1e-3))
learn.save('stage-2')

# Export

In [ ]:
learn.export()

# Test

In [ ]:
learn = load_learner(path, "export.pkl")

In [ ]:
data = LoadData(path, bs, 256)
learn.data = data

In [ ]:
learn.validate(metrics=[accuracy, error_rate])

# Interpretation

In [ ]:
learn.load('stage-2-256-modified-wd')
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=2)

# Test

In [ ]:
def Predict(imageList, count):
    results = []
    for img in imageList:
        _,_,outputs = learn.predict(img)
        percentages = [float(i) for i in outputs]
        results.append(percentages)
    
    averages = np.mean(results, axis=0)
    percentages = [ int(i * 100) for i in averages]
    values = sorted(set(zip(learn.data.classes, percentages)), key = lambda d: d[1], reverse=True)[:count]
    #total = sum(int(v) for n,v in values)
    #values.append(("other", float(100 - total)))
    return values

In [ ]:
images = [ open_image(path) for path in Path("./data/test/fig").ls() ]
Predict(images, 3)

In [ ]:
images = [ open_image(path) for path in Path("./data/test/jack").ls() ]
Predict(images, 3)